In [1]:
import aiida

In [2]:
from aiida.manage.configuration import get_config

get_config()

In [3]:
import os
import io
import json      # 🚩 补上这个
import zipfile   # 🚩 补上这个
from pathlib import Path
from aiida import load_profile, orm
from aiida.orm import Group, Node, QueryBuilder, ProcessNode, Node
from aiida.manage.configuration import get_config
from aiida.manage.manager import get_manager
from aiida.storage.sqlite_zip.backend import SqliteZipBackend
from aiida.manage import Profile
def load_archive_profile(filepath: str):
    """
    将压缩包作为临时 Profile 加载（主要用于 AiiDA 2.x 的只读探测）。
    """
    try:
        from aiida.storage.sqlite_zip.backend import SqliteZipBackend
        archive_profile = SqliteZipBackend.create_profile(filepath = filepath)
        profile = load_profile(archive_profile, allow_switch=True)
        # 这里的实现取决于你的具体环境配置，通常建议直接通过 get_archive_info 探测
        # 如果需要完整加载，通常使用临时存储后端
        return profile
    except Exception as e:
        raise Warning(f"Error loading archive: {e}")
def get_recent_processes(limit: int = 5):
    """
    🚩 核心：封装 AiiDA 数据库查询逻辑。
    这个函数既可以给 AI 当 Tool 用，也可以给 Controller 当内部数据源用。
    """
    qb = QueryBuilder()
    qb.append(ProcessNode, project=['id', 'attributes.process_state', 'attributes.process_label', 'ctime'], tag='process')
    qb.order_by({'process': {'ctime': 'desc'}})
    qb.limit(limit)
    
    results = []
    for pk, state, label, ctime in qb.all():
        results.append({
            'pk': pk,
            'state': state.value if hasattr(state, 'value') else str(state),
            'label': label or 'Unknown Task'
        })
    return results
def load_archive_profile(filepath: str):
    """
    将压缩包作为临时 Profile 加载（主要用于 AiiDA 2.x 的只读探测）。
    """
    try:
        from aiida.storage.sqlite_zip.backend import SqliteZipBackend
        archive_profile = SqliteZipBackend.create_profile(filepath = filepath)
        profile = load_profile(archive_profile, allow_switch=True)
        # 这里的实现取决于你的具体环境配置，通常建议直接通过 get_archive_info 探测
        # 如果需要完整加载，通常使用临时存储后端
        return profile
    except Exception as e:
        raise Warning(f"Error loading archive: {e}")
def get_statistics(profile_name: str = None):
    """
    获取数据库的高层统计信息。
    """
    if profile_name:
        switch_profile(profile_name)
            
    output = io.StringIO()
    output.write(f"=== Database Stats ({get_manager().get_profile().name}) ===\n")
    
    types = {
        "Calculations": "process.calculation.calcjob.CalcJobNode.",
        "WorkChains": "process.workflow.workchain.WorkChainNode.",
        "Structures": "data.core.structure.StructureData."
    }
    
    for name, node_type in types.items():
        count = QueryBuilder().append(Node, filters={"node_type": {"like": f"{node_type}%"}}).count()
        output.write(f"{name}: {count}\n")
        
    return output.getvalue()
def list_system_profiles():
    """
    获取系统中所有 AiiDA Profile 的名称列表。
    (修复了感知器找不到该函数的问题)
    """
    config = get_config()
    try:
        return [p.name for p in config.profiles]
    except Exception as e:
        raise Warning(f"AiiDA config not found or invalid: {e}")
        return []

In [5]:

load_profile()
load_archive_profile(r'C:\Users\Administrator\Desktop\Yiming\sab-core\engines\aiida\data\archives\test_dislocation.aiida')
print(get_statistics())
qb = QueryBuilder().append(
    orm.Node, 
    filters={
        "node_type": {"like": "process.workflow.workchain.WorkChainNode."}
        },
    project=['id', 'attributes.process_state', 'attributes.process_label', 'ctime'], tag='process')
qb.all()


Warning: Error loading archive: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmpoh6pj4zt'